In [4]:
#bean leaf lessions classification

In [3]:
!pip install opendatasets --quiet
import opendatasets as od
od.download("https://www.kaggle.com/datasets/marquis03/bean-leaf-lesions-classification")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: nithesh1011
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/marquis03/bean-leaf-lesions-classification


100%|██████████| 155M/155M [00:00<00:00, 973MB/s]  

In [54]:
import numpy as np
import pandas as pd
from torch import nn
from torch.optim import Adam
from torchvision.transforms import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import torch
import os
import torchvision.models as models
from torchvision.models import GoogLeNet_Weights

In [55]:
train_df = pd.read_csv("/content/bean-leaf-lesions-classification/train.csv")
val_df = pd.read_csv("/content/bean-leaf-lesions-classification/val.csv")

In [56]:
base_path = "/content/bean-leaf-lesions-classification"

train_df.iloc[:, 0] = train_df.iloc[:, 0].apply(lambda x: os.path.join(base_path, x))
val_df.iloc[:, 0] = val_df.iloc[:, 0].apply(lambda x: os.path.join(base_path, x))


In [57]:
train_df.head()

,image:FILE,category
0,/content/bean-leaf-lesions-classification/trai...,0
1,/content/bean-leaf-lesions-classification/trai...,0
2,/content/bean-leaf-lesions-classification/trai...,0
3,/content/bean-leaf-lesions-classification/trai...,0
4,/content/bean-leaf-lesions-classification/trai...,0


In [58]:
val_df.head()

,image:FILE,category
0,/content/bean-leaf-lesions-classification/val/...,0
1,/content/bean-leaf-lesions-classification/val/...,0
2,/content/bean-leaf-lesions-classification/val/...,0
3,/content/bean-leaf-lesions-classification/val/...,0
4,/content/bean-leaf-lesions-classification/val/...,0


In [59]:
train_df['category'].unique()

array([0, 1, 2])

In [60]:
print(train_df.shape)
print(val_df.shape)

(1034, 2)
(133, 2)


In [61]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_df.iloc[:, 1] = le.fit_transform(train_df.iloc[:, 1])
val_df.iloc[:, 1] = le.transform(val_df.iloc[:, 1])


In [62]:
transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.ConvertImageDtype(torch.float),
])

In [63]:
class CustomImageDataset(Dataset):
    def __init__(self, dataframe, transform):
        self.dataframe = dataframe
        self.transform = transform

    def __getitem__(self, index):
        img_path = self.dataframe.iloc[index, 0]
        label = self.dataframe.iloc[index, 1]  # category column
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, torch.tensor(label, dtype=torch.long)

    def __len__(self):
        return len(self.dataframe)

In [64]:
train_dataset = CustomImageDataset(train_df, transform=transform)
val_dataset = CustomImageDataset(val_df, transform=transform)

In [65]:
LR = 1e-3
BATCH_SIZE = 4
EPOCHS = 15

In [66]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [67]:
googlenet_model = models.googlenet(weights=GoogLeNet_Weights.DEFAULT)

In [68]:
num_classes = len(train_df["category"].unique())
googlenet_model.fc = nn.Linear(googlenet_model.fc.in_features, num_classes)
googlenet_model.to(device)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [69]:
loss_fun = nn.CrossEntropyLoss()
optimizer = Adam(googlenet_model.parameters(), lr=LR)

In [70]:
for epoch in range(EPOCHS):
    googlenet_model.train()
    total_loss_train = 0
    total_acc_train = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = googlenet_model(inputs)
        loss = loss_fun(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss_train += loss.item()
        total_acc_train += (outputs.argmax(1) == labels).sum().item()

    total_loss_train /= len(train_loader)
    total_acc_train = total_acc_train / len(train_dataset)

    print(f"Epoch [{epoch+1}/{EPOCHS}] "
          f"Train Loss: {total_loss_train:.4f}, "
          f"Train Acc: {total_acc_train*100:.2f}%")

Epoch [1/15] Train Loss: 0.7924, Train Acc: 66.44%
Epoch [2/15] Train Loss: 0.5936, Train Acc: 77.66%
Epoch [3/15] Train Loss: 0.5114, Train Acc: 80.27%
Epoch [4/15] Train Loss: 0.4638, Train Acc: 84.33%
Epoch [5/15] Train Loss: 0.3341, Train Acc: 87.81%
Epoch [6/15] Train Loss: 0.3021, Train Acc: 89.85%
Epoch [7/15] Train Loss: 0.2756, Train Acc: 91.68%
Epoch [8/15] Train Loss: 0.2178, Train Acc: 93.13%
Epoch [9/15] Train Loss: 0.2248, Train Acc: 91.59%
Epoch [10/15] Train Loss: 0.2350, Train Acc: 91.88%
Epoch [11/15] Train Loss: 0.1745, Train Acc: 94.39%
Epoch [12/15] Train Loss: 0.1536, Train Acc: 95.65%
Epoch [13/15] Train Loss: 0.2492, Train Acc: 91.59%
Epoch [14/15] Train Loss: 0.2154, Train Acc: 93.13%
Epoch [15/15] Train Loss: 0.0979, Train Acc: 97.29%


In [71]:
def test_model(model, test_loader, loss_fun, device):
    model.eval()
    total_loss, total_correct = 0, 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = loss_fun(outputs, labels)

            total_loss += loss.item()
            total_correct += (outputs.argmax(1) == labels).sum().item()

    avg_loss = total_loss / len(test_loader)
    avg_acc = total_correct / len(test_loader.dataset)

    print(f"Test Loss: {avg_loss:.4f}, Test Acc: {avg_acc*100:.2f}%")
    return avg_loss, avg_acc


In [73]:
test_loss, test_acc = test_model(googlenet_model, val_loader, loss_fun,'cuda')


Test Loss: 0.1453, Test Acc: 93.98%
